<a href="https://colab.research.google.com/github/ClaireJ59/ClaireJ59.github.io/blob/main/BackGround_Remove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安裝必要套件
!pip install rembg onnxruntime pillow --quiet


In [ ]:
import os
from PIL import Image
from rembg import remove, new_session
from google.colab import files
import io
import zipfile

# 建立輸出資料夾
os.makedirs("outputs", exist_ok=True)

# 工具函數：清理 alpha 邊緣
def clean_alpha(img, threshold=180):
    r, g, b, a = img.split()
    a = a.point(lambda p: 255 if p >= threshold else 0)
    return Image.merge("RGBA", (r, g, b, a))

# 工具函數：混白底避免黑邊
def remove_alpha_halo(img, bg_color=(255, 255, 255)):
    bg = Image.new("RGBA", img.size, bg_color + (255,))
    return Image.alpha_composite(bg, img).convert("RGB")

# 工具函數：根據檔名開頭取得縮放比例
def get_scale_ratio(filename):
    if filename.startswith("[L]"):
        return 0.95
    elif filename.startswith("[M]"):
        return 0.85
    elif filename.startswith("[S]"):
        return 0.75
    else:
        return 0.85  # 預設比例

# 單張圖片處理
def process_image(file_name, session):
    base_name = os.path.splitext(os.path.basename(file_name))[0]
    ratio = get_scale_ratio(base_name)

    # 開啟圖片並去背
    original = Image.open(file_name).convert("RGBA")
    with io.BytesIO() as buf:
        original.save(buf, format="PNG")
        removed_bytes = remove(buf.getvalue(), session=session)
    removed_img = Image.open(io.BytesIO(removed_bytes)).convert("RGBA")

    # 清除殘影
    removed_img = clean_alpha(removed_img, threshold=180)

    # 主體裁切
    alpha = removed_img.split()[3]
    bbox = alpha.getbbox()
    cropped = removed_img.crop(bbox)

    # 計算縮放目標尺寸
    canvas_size = (500, 667)
    target_width = int(canvas_size[0] * ratio)
    target_height = int(canvas_size[1] * ratio)
    cropped.thumbnail((target_width, target_height), Image.LANCZOS)

    # 混白底避免黑邊
    flattened = remove_alpha_halo(cropped)

    # 建立白底畫布並置中貼上
    background = Image.new("RGB", canvas_size, (255, 255, 255))
    x = (canvas_size[0] - flattened.size[0]) // 2
    y = (canvas_size[1] - flattened.size[1]) // 2
    background.paste(flattened, (x, y))

    # 儲存
    output_path = f"outputs/{base_name}.jpg"
    background.save(output_path, "JPEG", quality=95)
    return output_path

# 上傳多張圖片
uploaded = files.upload()

# 初始化去背模型
session = new_session("u2netp")

# 處理所有圖片
all_outputs = []
for file_name in uploaded.keys():
    final_path = process_image(file_name, session)
    all_outputs.append(final_path)

# 打包為 ZIP
zip_name = "outputs.zip"
with zipfile.ZipFile(zip_name, "w") as zipf:
    for file in all_outputs:
        zipf.write(file)

# 提供下載
files.download(zip_name)


Saving [L]1743246937498.jpg to [L]1743246937498 (2).jpg
Saving [S]1743259777350.jpg to [S]1743259777350 (2).jpg
Saving [L]1743246937354.jpg to [L]1743246937354 (2).jpg
Saving [M]1743246871656.jpg to [M]1743246871656 (2).jpg


100%|█████████████████████████████████████| 4.57M/4.57M [00:00<00:00, 4.72GB/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>